In [5]:
import pandas as pd
import numpy as np 
import sys

This module is for 

1. extracting the top two most frequently used OD's before and after new line
2. extracting the top two most frequent departure/arrival times before and after new line

by using Tuesday to Thursday data before and after new line

Dates used for data before new line: 2015/3/3-5

Dates used for data after new line: 2015/4/28-30

=============================================================================

New line was opened on April 1st of 2015

### read in data

In [17]:
# before = ['2015030{}'.format(day) for day in range(3,6)]
# after = ['201504{}'.format(day) for day in range(28,31)]
before = ['20150304']
after = ['20150429']

header = pd.read_csv("../nanjing-data/header.csv")

df = {}
for date in before + after:
    
    # attach header
    df[date] = pd.read_csv('../nanjing-data/{}.csv'.format(date), header=None)
    df[date].columns = header.columns

    # drop unnecessary columns
    df[date].drop(labels=['Card_Id_Long', 'Card_Type', 'Device_Id'], axis=1, inplace=True)

    # convert datetime
    df[date]['Time'] = pd.to_datetime(df[date]['Time'])
    
    df[date].sort_values('Time', inplace=True)
    
df[before[0]].head()

,Card_Id,Fare,Time,Entry_Or_Exit,Line_Id,StationId
1403858,996260140165,0.0,2015-03-04 01:43:22,1,2,27
547408,996260138943,0.0,2015-03-04 02:05:19,1,2,25
711162,996260138943,0.0,2015-03-04 02:05:35,2,2,25
248347,996260140434,0.0,2015-03-04 02:12:17,2,3,46
1594810,996260140979,0.0,2015-03-04 02:23:19,2,2,29


In [24]:
# consider only IDs present in both before and after days

ids_before = set()
ids_after = set()
for i in range(len(before)):
    ids_before = ids_before.union(set(df[before[i]]['Card_Id'].unique()))
    print(len(ids_before))
    ids_after = ids_after.union(set(df[after[i]]['Card_Id'].unique()))
    print(len(ids_after))
ids = np.array(list(ids_before.intersection(ids_after)))
ids.shape

476095
580141


(208636,)

In [25]:
# pick out a sample of IDs
n = 10_000
sample_inds = np.random.choice(len(ids), size=n, replace=False)
ids = ids[sample_inds]
len(ids)

10000

In [26]:
# drop rows of card ids not in before or after
print(df[before[0]].shape)
for date in before + after:
    df[date] = df[date].loc[df[date]['Card_Id'].isin(ids),:]
print(df[before[0]].shape)

(1660000, 6)
(36876, 6)


In [37]:
# convert rows of entry and exit into OD pairs with fare

od = {}
for date in before + after:
    od[date] = pd.DataFrame({'id':[], 'station1':[], 'station2':[], 'fare':[]})
    temp = {}
    for i,row in df[date].iterrows(): # for row in each day's trips
        if row['Entry_Or_Exit'] == 1: # this is an entry, open a trip
            temp[row['Card_Id']] = row['StationId']
        elif row['Entry_Or_Exit'] == 2: # implicitly drop bad data (not 1 or 2)
            if row['Card_Id'] in temp:
                newrow = {}
                newrow['id'] = row['Card_Id']
                newrow['station1'] = temp[row['Card_Id']]
                newrow['station2'] = row['StationId']
                newrow['fare'] = row['Fare']
                od[date].loc[od[date].shape[0]] = newrow
                del(temp[row['Card_Id']])
od[before[0]].head()

,fare,id,station1,station2
0,0.0,9.962641e+11,10.0,10.0
1,0.0,9.962601e+11,31.0,31.0
2,0.0,9.962641e+11,42.0,42.0
3,0.0,9.960610e+11,10.0,12.0
4,0.0,9.971667e+11,13.0,14.0


In [48]:
# export result
for date in before:
    od[date].to_csv('ods_b4_{}.csv'.format(date), index=False)
for date in after:
    od[date].to_csv('ods_af_{}.csv'.format(date), index=False)

In [50]:
# determine which IDs switched lines
l3ids = {73, # new stations on line 3
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113}

# switch_or_not = np.zeros((n,2))
# switch_or_not[:,0] = ids
# for i,uid in enumerate(ids):
#     # figure out if uid has traveled on line 3 afterward
#     for date in after:
#         select = od[date].loc[od[date]['id'] == uid,:]
#         if (select['station1'].isin(l3ids).any()) or (select['station2'].isin(l3ids).any()):
#             switch_or_not[i,1] = 1
# switch_or_not[0:10,:]

switch_or_not = pd.DataFrame({'Card_Id':ids, 'switched':np.zeros(n)})
for i,uid in enumerate(ids):
    # figure out if uid has traveled on line 3 afterward
    for date in after:
        select = od[date].loc[od[date]['id'] == uid,:]
        if (select['station1'].isin(l3ids).any()) or (select['station2'].isin(l3ids).any()):
            switch_or_not.iloc[i,1] = 1
switch_or_not.head()

,Card_Id,switched
0,993171075763,0.0
1,997168924030,0.0
2,990163953659,0.0
3,976674428973,0.0
4,993171087015,0.0


In [51]:
# export switching
# np.savetxt('switch_or_not.csv',switch_or_not,delimiter=',')
switch_or_not.to_csv('switch_or_not.csv', index=False)